## siRNA knockdown fixed ##
This notebook fits traces with fixed mRNA expression.

An amount of mRNA is added to the cells.
At a certain time $t=0$, a fixing agent (CHX) is added, and mRNA expression is stopped.
Now, an initial amount $G_{u0}$ of pre-mature protein and an initial amount $G_0$ of mature protein is in the cell.

After adding the fixing agent, the pre-mature protein can mature with maturation rate $k_m$, and both pre-mature and mature protein can degrade with degradation rate $\beta$.

This system can be described by the following differential equations:
$$\begin{align*}
\frac{\mathrm{d}G_u}{\mathrm{d}t} &= -\beta G_u - k_m G_u \\
\frac{\mathrm{d}G}{\mathrm{d}t} &= -\beta G + k_m G_u
\end{align*}$$

The solution for the amount of mature protein $G(t)$ is:
$$
G(t) = G_0 \mathrm{e}^{-\beta t} + G_{u0}\left(\mathrm{e}^{-\beta t} - \mathrm{e}^{-(\beta+k_m)t}\right)
$$
The parameters to fit are $\beta$, $k_m$ and $G_{u0}$.

## Notebook structure
The notebook has the following structure:

At first, the model functions are defined and the data is loaded. The next section contains code for fitting the two models separately. The next section contains code for fitting the two traces in one run with parameters shared among the models.

Fitting requires that the result list `R` is defined, which can be done by running the corresponding cell. When `R` has been populated by fitting, the results can be plotted. There are cells for plotting the results of the separate fit, the results of the combined fit, and the pure parameter distributions of all fits.

Additionally, there are cells for saving and loading paramaters by python’s `pickle` module.

In [ ]:
# Import modules needed

# Standard library
import os
import pickle
import sys

# Scientific stack
import numpy as np
np.seterr(divide='print')
import pandas as pd
pd.set_option('display.max_rows', None)
#import scipy as sc
from scipy.cluster import hierarchy as ch

# Matplotlib
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42 # Make fonts editable by Adobe Illustrator
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

# Notebook utilities
import IPython
import ipywidgets as wdg

In [ ]:
# Define utility functions
def getTimeStamp():
    """Returns a human-readable string representation of the current time"""
    import time
    from datetime import datetime
    now = datetime.now()
    if os.name == 'nt':
        # Cope with unicode incapability of time library in Windows
        return '–'.join((now.strftime("%Y-%m-%d"), now.strftime("%H%M%S")))
    return now.strftime("%Y-%m-%d–%H%M%S")


def getOutpath(filename='', timestamp=None):
    """Returns (and creates, if necessary) the path to a directory
    called “out” inside the current directory.
    If `filename` is given, the filename is appended to the output directory.
    A timestamp will be added to the filename if `timestamp != ''`.
    If timestamp is `None`, the current timestamp is used.
    """
    # Create output directory
    outpath = os.path.join(os.getcwd(), 'out')
    if not os.path.isdir(outpath) and not os.path.lexists(outpath):
        os.mkdir(outpath)

    # If requested, build filename
    if len(filename) > 0:
        if timestamp == None:
            timestamp = getTimeStamp()
        outpath = os.path.join(outpath, ((timestamp + '_') if len(timestamp) > 0 else '') + filename)
    return outpath


def getDataLabel(i, isFilename=False):
    """Returns a nicely formatted name for the `i`-th element of `D`.
    Set `isFilename=True` for a filename-friendly output."""
    if isFilename:
        return "{0[measurement]}_{0[sample]}_{0[condition]}".format(D[i])
    return "{0[sample]}: {0[condition]} [{0[measurement]}]".format(D[i])

## Read in data and prepare result list

In [ ]:
# Prepare data loading

# Define available files
datafiles = [
    {
        "sample": "Huh7",
        "condition": "rfp",
        "measurement": "2017-09-08_seq6",
        "file": "data/2017-09-08_seq6_Huh7_CayRFP_CHX_7h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "rfp",
        "measurement": "2017-09-08_seq7",
        "file": "data/2017-09-08_seq7_Huh7_CayRFP_CHX_7h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "rfp",
        "measurement": "2017-09-08_seq8",
        "file": "data/2017-09-08_seq8_Huh7_CayRFP_CHX_5h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "rfp",
        "measurement": "2017-09-08_seq9",
        "file": "data/2017-09-08_seq9_Huh7_CayRFP_CHX_5h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "rfp",
        "measurement": "2017-09-08_seq10",
        "file": "data/2017-09-08_seq10_Huh7_CayRFP_CHX_3h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "rfp",
        "measurement": "2017-09-08_seq11",
        "file": "data/2017-09-08_seq11_Huh7_CayRFP_CHX_3h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "gfp",
        "measurement": "2017-08-18_seq6",
        "file": "data/2017-08-18_seq6_Huh7_eGFP_CHX_7h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "gfp",
        "measurement": "2017-08-18_seq7",
        "file": "data/2017-08-18_seq7_Huh7_eGFP_CHX_7h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "gfp",
        "measurement": "2017-08-18_seq8",
        "file": "data/2017-08-18_seq8_Huh7_eGFP_CHX_5h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "gfp",
        "measurement": "2017-08-18_seq9",
        "file": "data/2017-08-18_seq9_Huh7_eGFP_CHX_5h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "gfp",
        "measurement": "2017-08-18_seq10",
        "file": "data/2017-08-18_seq10_Huh7_eGFP_CHX_3h.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "gfp",
        "measurement": "2017-08-18_seq11",
        "file": "data/2017-08-18_seq11_Huh7_eGFP_CHX_3h.xlsx"
    }
]

# By default, mark all files for loading
load_idcs = range(len(datafiles))

# Define function for loading data
def load_data_from_files():
    """Loads data from specified files into `D`.
    Requires `load_idcs` to hold a list of indices to `datafiles`."""
    global D
    D = []
    for i in load_idcs:
        # Show message
        print("Loading file: {}".format(datafiles[i]["file"]))

        # Read sheets from excel file
        X = pd.read_excel(datafiles[i]['file'], dtype=np.float64, sheet_name=0)

        # Write data into easy-to-access structure
        d = {}
        d['sample'] = datafiles[i]['sample']
        d['condition'] = datafiles[i]['condition']
        d['measurement'] = datafiles[i]['measurement']
        d['file'] = datafiles[i]['file']

        d['t'] = X.values[:,0].flatten()
        d['fluorescence'] = X.values[:,1:]
        D.append(d)

In [ ]:
# Read in data from excel sheets

# Prompt user for files to load
lbl = wdg.Label('Select the files to load:')
lbl.layout.width = 'initial'
entries = []
for f in datafiles:
    entries.append("{} {}: {}".format(
        f['sample'], f['condition'], f['file']))
sel_entry = wdg.SelectMultiple(options=entries, rows=len(entries))
sel_entry.layout.width = 'initial'
bload = wdg.Button(description='Load')
bselall = wdg.Button(description='Select all')
bselnone = wdg.Button(description='Select none')

# Define callbacks
def sel_all_files(_):
    sel_entry.value = entries
def sel_no_files(_):
    sel_entry.value = ()
def load_button_clicked(_):
    global load_idcs
    load_idcs = [entries.index(r) for r in sel_entry.value]
    vb.close()
    load_data_from_files()
bselall.on_click(sel_all_files)
bselnone.on_click(sel_no_files)
bload.on_click(load_button_clicked)

# Finally, show the widgets
vb = wdg.VBox((lbl, sel_entry, wdg.HBox((bload,bselall,bselnone))))
IPython.display.display(vb)

In [ ]:
# Provide output tables

# Initialize result list
R = []

# Iteratively populate the result dictionary
for k in range(len(D)):
    nTimes, nTraces = np.shape(D[k]['fluorescence'])
    R.append(np.full((nTraces,), np.NaN, dtype=[('t0', 'f8'), ('nInit', 'u2')]))

### Pickle or load fitting results
Pickling is only reasonable if the result list `R` has already been populated by fitting (see below).

In [ ]:
# Pickle fit results for future sessions
outfile = getOutpath('fixed_fit_results.pickled')
with open(outfile, 'wb') as f:
    pickle.dump(R, f)

In [ ]:
# Load pickled results (requires file suffix “.pickled”)
pickfiles = [f for f in os.listdir(getOutpath()) if f.lower().endswith('.pickled')]
pickfiles.sort(reverse=True)

lbl = wdg.Label('Select the file to load:')
lbl.layout.width = 'initial'
rad = wdg.RadioButtons(options=pickfiles)
but = wdg.Button(description='Load')
vb = wdg.VBox([lbl, rad, but])
IPython.display.display(vb)

def clicked_on_but(b):
    global R
    with open(getOutpath(rad.value, ''), 'rb') as f:
        R = pickle.load(f)
    print('Loaded: ' + rad.value)
    vb.close()
but.on_click(clicked_on_but)

In [ ]:
# Write results to XLSX
if len(R) != len(D):
    raise ValueError("R and D must have the same length!")

# Set up XLSX writer
file = getOutpath("CHX_onset_cluster.xlsx")
xlsx_writer = pd.ExcelWriter(file, engine='xlsxwriter')

# Write each datafile into its sheet
for k in range(len(D)):
    datalabel = getDataLabel(k, True)
    pd.DataFrame(R[k]).to_excel(xlsx_writer, sheet_name=datalabel, na_rep='NaN')
    
xlsx_writer.save()

## Find onset by clustering

In [ ]:
def get_Z(time, data, isTimeNormalized=False, time_scale=20):
    """Finds the onset time by legacy algorithm. Not robust on new data.

    Input parameters:
        time: numpy vector of times of datapoints (in hours)
        data: numpy vector of fluorescence intensity
        isTimeNormalized: boolean indicating wheter `time` is normalized already
        time_scale: scaling factor by which to divide the normalized time

    Return value:
        Z: the cluster tree returned by `linkage`
    """

    # Normalized time (in hours/`time_scale`)
    if not isTimeNormalized:
        tn = (time - time.min()) / (time.max() - time.min()) / time_scale
    else:
        tn = time

    # Normalized data
    dn = (data - data.min()) / (data.max() - data.min())

    # Gradient of normalized data
    gradn = np.empty(np.shape(dn))
    gradn[1:] = (dn[1:] - dn[:-1]) / (time[1:] - time[:-1])
    gradn[0] = gradn[1]
    gradn[gradn < 0] = 0

    # Combine to observation matrix
    tab = np.array([tn, dn, gradn]).T

    # Perform clustering
    Z = ch.linkage(tab, method='single', metric='cityblock')
    return Z


def get_t0(Z, time, height=0.025):
    """Finds the onset time and the clusters before it by the legacy algorithm

    Input parameters:
        Z: the cluster tree returned by `linkage`
        time: numpy vector of times of datapoints (in hours)
        height: threshold at which to cut cluster tree

    Return value:
        t0: the onset time found
    """
    # Find last time point in initial cluster
    ct = ch.cut_tree(Z, height=height).flatten()
    t0 = time[ct == ct[0]][-1]
    return t0


def getBranchTime(b, Z, time):
    """Returns the time corresponding to a cluster node
        b: node index (zero-based)
        Z: cluster tree as returned from scipy.cluster.hierarchy.linkage
        time: the time vector of measurements"""
    nLeaves = time.size
    b = int(b)
    if b < nLeaves:
        return time[b]
    else:
        b -= nLeaves
        t1 = getBranchTime(Z[b,0], Z, time)
        t2 = getBranchTime(Z[b,1], Z, time)
        return (t1 + t2) / 2


def climb_up(c, Z):
    """Climbs cluster hierarchy up by one step.

    Input parameters:
        c: cluster to start from
        Z: cluster tree (from `linkage`)

    Returns:
        index of the cluster containing cluster `c`, or empty array if no cluster found
    """
    return np.flatnonzero(np.any(Z[:,:2] == c, axis=1)) + Z.shape[0] + 1


def climb_down(c, Z):
    """Climbs cluster hierarchy down by one step.

    Input parameters:
        c: cluster to start from
        Z: cluster tree (from `linkage`)

    Returns:
        array of indices of child clusters, or empty array if no child clusters found
    """
    c -= Z.shape[0] + 1
    if c < 0:
        return np.empty((0), dtype=np.intp)
    else:
        return Z[c,:2].astype(np.intp)


def initial_clusters(Z, height=0.025):
    """Returns a boolean array indicating nodes belonging to initial cluster

    Input parameters:
        Z: cluster tree (from `linkage`)
        height: maximum height of initial cluster

    Returns:
        boolean array of size `Z.shape[0] * 2 + 1`, such that the i-th value is
        `True` iff the i-th cluster belongs to the initial cluster
    """
    # Initialize variables
    nLeaves = Z.shape[0] + 1
    isInitial = np.zeros((nLeaves * 2 - 1), dtype=np.bool_)
    maxInitial = np.zeros(1, dtype=np.intp)

    # Climb up to highest node in initial cluster
    while True:
        newMax = climb_up(maxInitial, Z)
        if newMax.size == 0:
            break
        elif Z[newMax - nLeaves, 2] > height:
            break
        else:
            maxInitial = newMax

    # Recursively set entries for all nodes in initial cluster to `True`
    inits = maxInitial.tolist()
    for i in inits:
        isInitial[i] = True
        inits.extend(climb_down(i, Z).tolist())

    return isInitial

In [ ]:
# Get onset times by clustering

def clustering(time, data, height=0.025, iTrace=None, dataset=None, pdf=None, isShow=True):
    """
    Find t0 and plot to PDF
    """
    # Find t0 by clustering
    Z = get_Z(time, data)
    t0 = get_t0(Z, time, height=height)

    # Write array indicating initial cluster
    isInitial = initial_clusters(Z, height=height)
    nInitial = np.sum(isInitial, dtype=np.uint16)

    # Create plot if requested
    if isShow or pdf is not None:
        nLeaves = time.size

        # Create figure to plot to
        fig = plt.figure()
        ax1 = fig.add_subplot(2, 1, 1)
        ax2 = fig.add_subplot(2, 1, 2, sharex=ax1)
        fig.subplots_adjust(hspace=0, wspace=0)

        # Indicate onset time
        ax1.axvline(t0, color='r')
        ax2.axvline(t0, color='r')

        # Indicate maximum height of initial cluster
        ax2.axhline(height, color='k', ls='--', lw=.5)

        # Plot trace with t0
        ax1.plot(time, data, '-k')

        # Get minimum of trace
        trace_min = data.min()

        # Plot vertical lines below graph
        for t in range(nLeaves):
            if isInitial[t]:
                clr = 'm'
            else:
                clr = 'b'
            ax1.plot([time[t], time[t]], [data[t], trace_min],
                     '-', color=clr, linewidth=.5, clip_on=False)
            ax1.plot(time[t], data[t],
                    'o', color=clr, markersize=2, clip_on=False)

        # Plot custom dendrogram because built-in dendrogram does
        # not allow for custom ordering of leaves
        for z in range(Z.shape[0]):
            b1, b2 = (Z[z,0], Z[z,1]) if Z[z,0] < Z[z,1] \
                else (Z[z,1], Z[z,0])
            b1 = int(b1)
            b2 = int(b2)
            h1 = 0 if b1 < nLeaves else Z[b1-nLeaves,2]
            h2 = 0 if b2 < nLeaves else Z[b2-nLeaves,2]
            t1 = getBranchTime(b1, Z, time)
            t2 = getBranchTime(b2, Z, time)

            node = np.empty((4, 2))
            node[0,:] = [t1, h1]
            node[1,:] = [t1, Z[z,2]]
            node[2,:] = [t2, Z[z,2]]
            node[3,:] = [t2, h2]

            if isInitial[z + nLeaves]:
                clr = 'm'
            else:
                clr = 'b'
            ax2.plot(node[:,0], node[:,1], '-',
                     color=clr, linewidth=.5)

        # Format figure
        ax1.tick_params('x', bottom=False, labelbottom=False)
        ax1.set_ylim(bottom=trace_min)

        ax2.invert_yaxis()
        ax2.set_ylim(top=0)
        ax2.set_yscale('symlog', linthreshy=0.001, linscaley=.5)
        ax1.set_axisbelow(True)
        ax2.set_axisbelow(True)

        ax2.set_xlabel('Time [h]')
        ax1.set_ylabel('Fluorescence [a.u.]')
        ax2.set_ylabel('Distance [a.u.]')

        suptitle = []
        if dataset is not None:
            suptitle.append(dataset)
        if iTrace is not None:
            suptitle.append("Trace {:03d}".format(iTr))
        fig.suptitle('\n'.join(suptitle))

        if pdf is not None:
            pdf.savefig(fig)
        if isShow:
            plt.show(fig)
        plt.close(fig)
        fig.clear()

    return t0, nInitial

In [ ]:
# Get onset times by clustering
ts = getTimeStamp()
isShow = 0

for iD, d in enumerate(D):
    datalabel = getDataLabel(iD)
    time = d['t']
    data = d['fluorescence']
    with PdfPages(getOutpath("ONSET_CLUSTER_{}.pdf".format(getDataLabel(iD, True)), ts)) as pdf:
        for iTr in range(data.shape[1]):
            t0, nInitial = clustering(time, data[:,iTr], iTrace=iTr, dataset=datalabel,
                                      pdf=pdf, isShow=True)

            # Save to results list
            R[iD][iTr] = t0, nInitial

            # Avoid notebook crash by too many figures
            if isShow:
                if isShow < 200:
                    isShow += 1
                else:
                    isShow = False

## Identify ill-shaped raw traces
For these measurements, the parts before CHX addition were cut away from the traces.
While most traces now start with an ascend, some traces have minima after the first value.
As our fitting model does not account for this behaviour, such traces may be fitted badly and affect the mean parameter values.

We therefore want to identify those traces and sort them out.

A trace is treated as ill-shaped if its smallest value before the maximum is
* the third or a later value or
* the second value and
  * its difference to the first value is larger than 3% of its difference to the maximum or
  * another value before the maximum is smaller than the first value.

In [ ]:
def find_bad_traces(data):
    """
    Finds ill-shaped traces. A trace is ill-shaped or bad if
    any value before the maximum is smaller than the first value.
    However, if only the second value is smaller than the first value
    and the difference between first and second value does not exceed
    a threshold (3%) of the difference between maximum value and
    minimum value, the trace is not assumed bad.

    Argument:
        data -- array of traces (columns: traces, row: timepoints)

    Returns:
        1-dim indexing array, where the i-th element indicates if
        the i-th trace from data is bad (True) or not (False).
    """
    # Find maxima and minima of the traces
    maxima = data.argmax(axis=0)
    minima = np.zeros_like(maxima)
    for i, m in enumerate(maxima):
        minima[i] = data[:m,i].argmin(axis=0)

    # Mark all traces as bad where the first value is not the minimum
    bad_traces = minima > 0

    for i, m in enumerate(minima):
        # If only the second value is smaller than the first …
        if (m == 1) and data[2:maxima[i],i].min() > data[0,i]:
            # … check if the relative difference exceeds a threshold …
            amp = data[maxima[i],i] - data[m,i]
            if (data[0,i] - data[1,i]) / amp <= 0.3:
                # … and if not, do not discard the trace
                bad_traces[i] = False

    return bad_traces

In [ ]:
# Search for bad traces
bad_traces_red = pd.DataFrame(False, index=chsq_red.index, columns=('bad',), dtype=np.bool_)
bad_traces_green = pd.DataFrame(False, index=chsq_green.index, columns=('bad',), dtype=np.bool_)

for i, d in enumerate(D):
    condition = d['condition']
    data = d[condition]

    # Identify bad traces:
    # the minimum is before the maximum, but not the first point
    #minima = data.argmin(axis=0)
    #maxima = data.argmax(axis=0)
    #bad_traces = (minima > 0) & (minima < maxima)
    bad_traces = find_bad_traces(data)

    # Save bad traces in condition-specific table
    if condition == 'rfp':
        bad_traces_red.loc[i, 'bad'] = bad_traces
    elif condition == 'gfp':
        bad_traces_green.loc[i, 'bad'] = bad_traces

In [ ]:
# Plot bad traces
with PdfPages(getOutpath("fixed_bad_traces.pdf")) as pdf:
    for condition in ("rfp", "gfp"):
        if condition == 'rfp':
            bad_traces = bad_traces_red
        elif condition == 'gfp':
            bad_traces = bad_traces_green

        for i, j in bad_traces.index:
            if not bad_traces.loc[(i,j),'bad']:
                continue

            clr = condition[0]
            t = D[i]['t']

            f, ax = plt.subplots(1, 1)
            ax.plot(t, D[i][condition][:,j], '-', color=clr, lw=1, label="measured")
            ax.plot(t, R[i][condition]['fit'][:,j], '-k', lw=0.5, label="best fit")
            ax.legend()
            ax.set_xlabel("Time [h]")
            ax.set_ylabel("Fluorescence [a.u.]")
            ax.set_title("Trace {:03d} in {}".format(j, getDataLabel(i)))

            f.tight_layout(pad=0)
            plt.show(f)
            pdf.savefig(f)
            plt.close(f)